# Malicious Service Install
This method detects well-known keywords of malicious services in the Windows System Eventlog

## Rule Content
```
- action: global
  title: Malicious Service Install
  id: 4976aa50-8f41-45c6-8b15-ab3fc10e79ed
  description: This method detects well-known keywords of malicious services in the
    Windows System Eventlog
  author: Florian Roth
  tags:
  - attack.credential_access
  - attack.t1003
  - attack.s0005
  logsource:
    product: windows
    service: system
    category: null
  detection:
    selection1:
      EventID:
      - 7045
    keywords:
      Message:
      - '*WCE SERVICE*'
      - '*WCESERVICE*'
      - '*DumpSvc*'
    quarkspwdump:
      EventID: 16
      HiveName: '*\AppData\Local\Temp\SAM*.dmp'
    condition: ( selection1 and keywords ) or ( selection2 and keywords ) or quarkspwdump
  falsepositives:
  - Unlikely
  level: high
- logsource:
    product: windows
    service: security
  detection:
    selection2:
      EventID: 4697

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-system-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((Message.keyword:(*WCE\ SERVICE* OR *WCESERVICE* OR *DumpSvc*) AND (event_id:("7045") OR event_id:"4697")) OR (event_id:"16" AND hive_name.keyword:*\\AppData\\Local\\Temp\\SAM*.dmp))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()